Смотрите следующие репозитории для получения дополнительной информации:
https://github.com/TomuToyoshima/kaggle-child-mind-institute-detect-sleep-states


# Install libraries

In [ ]:
# Установка библиотек hydra-core и segmentation_models_pytorch версии 0.3.3
# Опция --no-index говорит pip не использовать индексы пакетов PyPI
# Опция --find-links указывает на локальный источник библиотек
!pip install hydra-core segmentation_models_pytorch==0.3.3 --no-index --find-links=/kaggle/input/ex-library

In [ ]:
# Переход в директорию с репозиторием на Kaggle
%cd /kaggle/input/cmi-code/kaggle-child-mind-institute-detect-sleep-states/

# Config

In [ ]:

# конст.
DURATION = 5760
DOWNSAMPLE_RATE = 2
PHASE = 'test'
EXP_NAME = 'exp001'

# Preprocess

In [ ]:

# Запуск скрипта подготовки данных с использованием Python
# Аргументы dir и phase определяют директорию и фазу обработки данных соответственно
!python -m run.prepare_data dir=kaggle phase=$PHASE

# Inference

- **Мой выбор `model.encoder_name`**: Я решил использовать архитектуру `resnet34`, но есть возможность изменить её на другую, например, на `resnet50` или `efficientnet_b3`. Это поможет адаптировать модель под конкретные задачи и эксперименты.

- **Изменение `model.encoder_weights`**: Сейчас я использую значение `null`, но есть опция выбрать другие предварительно обученные веса, если они доступны. Это может значительно улучшить результаты модели, особенно если она будет обучаться на данных, похожих на те, на которых были обучены эти веса.

- **Оптимизация `num_workers`**: Я установил значение равное 2, но это можно изменить, чтобы оптимизировать скорость обработки данных. Увеличение этого числа ускорит работу на многоядерных системах.

- **Настройка `batch_size`**: Важный параметр, который влияет на скорость обучения и инференса. Больший размер пакета ускоряет процесс, но требует больше оперативной памяти. Мне пришлось найти баланс между скоростью и доступным объемом памяти.

- **Параметры `duration` и `downsample_rate`**: Здесь я играю с продолжительностью и частотой понижения дискретизации данных, чтобы увидеть, как это влияет на входные данные модели. Изменение этих параметров может дать новые перспективы для анализа данных.

- **Настройка `post_process.score_th`**: Пороговое значение для обработки результатов. Я установил его довольно низко, чтобы уловить больше потенциальных событий, но его можно поднять для уменьшения ложных срабатываний.

- **Изменение `post_process.distance`**: Этот параметр определяет, насколько близко друг к другу должны быть события, чтобы считаться связанными. Изменение этого параметра помогает мне понять, как модель воспринимает последовательность событий.

- **Выбор `phase`**: Важно выбрать правильную фазу, в которой будет происходить инференс. У меня есть разные фазы для обработки данных, и изменение этого параметра позволяет мне адаптировать процесс к конкретной задаче.


In [ ]:
# feature extractor=CNN, encoder=UNet, decoder=UNet1D, downsample_rate=2, seq_len=5760

In [ ]:
# Запуск скрипта для выполнения вывода (инференса) модели
# Аргументы указывают на параметры модели, количество рабочих процессов, имя эксперимента,
# настройки весов, размер пакета, продолжительность, частоту понижения дискретизации,
# порог обработки результатов и расстояние между событиями, а также фазу обработки данных


#     model.encoder_name=resnet34\  # Можно изменить на другую модель, смотри описание
#     model.encoder_weights=null\   # Изменение весов модели, если доступны другие варианты
#     num_workers=2\                # Количество рабочих процессов, можно увеличить или уменьшить
#     exp_name=$EXP_NAME\           # Имя эксперимента, изменение влияет на логирование и сохранение результатов
#     weight.run_name=single\       # Настройка весов модели, можно выбрать другой вариант
#     batch_size=64\                # Размер пакета, увеличение ускорит процесс, но потребует больше памяти
#     duration=$DURATION\           # Продолжительность, изменение влияет на входные данные модели
#     downsample_rate=$DOWNSAMPLE_RATE\  # Частота понижения дискретизации, изменение влияет на обработку данных
#     post_process.score_th=0.005\  # Пороговое значение для обработки результатов, изменение влияет на чувствительность модели
#     post_process.distance=40\     # Расстояние для постобработки, влияет на интерпретацию последовательности данных
#     phase=$PHASE                  # Фаза обработки данных, изменение может влиять на результаты
#rye run python run/inference.py dir=kaggle exp_name=exp001 weight.run_name=single downsample_rate=2 
# duration=5760 model.params.encoder_weights=null
# pp.score_th=0.005
# pp.distance=40
# phase=test

# v1 0.005, 40 (0.707)
#v2 0.001, 50 (0.708)
#v3 0.005  90-100 (709)
#v4 0.005  95 (709)

!python -m run.inference\
    dir=kaggle\
    model.encoder_name=resnet34\
    model.encoder_weights=null\
    num_workers=2\
    exp_name=$EXP_NAME\
    weight.run_name=single\
    batch_size=256\
    duration=$DURATION\
    downsample_rate=$DOWNSAMPLE_RATE\
    post_process.score_th=0.005\
    post_process.distance=85\
    phase=$PHASE  







In [ ]:
# Удаление временных данных и результатов из рабочей директории на Kaggle
!rm -rf /kaggle/working/processed_data
!rm -rf /kaggle/working/output